> **Disclaimer**  
> This notebook uses content adapted from a course-provided notebook available at [github.com/langchain-ai/lca-langchainV1-essentials](https://github.com/langchain-ai/lca-langchainV1-essentials).  
> All original credit belongs to the course authors.

# Streaming

<img src="./assets/LC_streaming.png" width="400">

Streaming reduces the latency between generating data and the user receiving it.
There are two types frequently used with Agents:

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
load_dotenv()

from bedrock import nova_pro as llm

In [3]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    system_prompt="You are a full-stack comedian",
)

## No Streaming (invoke)

In [4]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

Sure, here's a joke for you:

Why did the full-stack developer go broke?

Because he used up all his cache! 💰💻

Hope that made you laugh! Remember, laughter is the best UI/UX. 😂


## values
You have seen this streaming mode in our examples so far. 

In [13]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a Dad joke
================================== Ai Message ==================================

Alright, here's a dad joke for you:

Why did the scarecrow win an award?

Because he was outstanding in his field! 🌾😄

Hope that brought a smile to your face!


## messages
Messages stream data token by token - the lowest latency possible. This is perfect for interactive applications like chatbots.

In [14]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):  
    if token.content:
        print(token.content[-1].get("text", ""), end="")

Sure, here's a family-friendly poem called "The Adventures of Sunny the Sun":

---

In the sky so high and wide,
Lives Sunny the Sun with a gleaming stride.
He wakes up each morn with a cheerful grin,
Ready to start his day, let the fun begin!

With a stretch and a yawn, he peeks out the door,
"Good morning, world!" he says, and much more.
He sends down his rays, warm and bright,
To greet every corner, bathing in light.

Sunny waves to the mountains, tall and grand,
And the oceans below, a shimmering strand.
He tickles the trees with his golden beams,
Waking up flowers from their sleepy dreams.

The birds chirp hello as they take to the sky,
Sunny winks at them, "Don't be in a hurry, fly!"
He paints the clouds in shades of white,
Creating fluffy castles, a delightful sight.

When the day grows long and the evening nears,
Sunny slows his dance, with no fears.
He dons a red and orange cloak so fine,
As he starts his descent, a beautiful decline.

He whispers goodnight to the moon up high

## Tools can stream too!
Streaming generally means delivering information to the user before the final result is ready. There are many cases where this is useful. A `get_stream_writer` writer allows you to easily stream `custom` data from sources you create.

In [17]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer #Used to stream tool data


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model=llm,
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='fdb4da70-9542-4495-ab1c-6342e0fea5fc')]})
('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='fdb4da70-9542-4495-ab1c-6342e0fea5fc'), AIMessage(content=[{'type': 'text', 'text': "<thinking> The user has asked for the weather in SF, which is commonly known as San Francisco. I need to use the 'get_weather' tool to fetch the current weather information for San Francisco. </thinking>\n"}, {'type': 'tool_use', 'name': 'get_weather', 'input': {'city': 'San Francisco'}, 'id': 'tooluse_BqjnTUrTRdCRpod0xlikjw'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '3aabfe6b-0de9-4576-8ed3-abb5140676c8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 04 Jan 2026 19:14:27 GMT', 'content-type': 'application/json', 'content-length': '523', 'connection': 'keep-alive', 'x-amzn-re

In [18]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

('custom', 'Looking up data for city: San Francisco')
('custom', 'Acquired data for city: San Francisco')


## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [19]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])

Looking up data for city: San Francisco
Acquired data for city: San Francisco
